# Workshop 2: Fundamentals of CNN

In this workshop we will learn how to implement a simple Convolutional Neural Netwrok and we will compare it with a Fully Connected Neural Network for the classification of CIFAR10 dataset. The structure of the workshop will be the following:



1.   CIFAR10 with Fully Connected Neural Netoworks
2.   CIFAR10 with Convolutional Neural Networks



## 1. CIFAR10 with Fully Connected Neural Networks

In [ ]:
# Import dependence for downloading CIFAR10
from tensorflow import keras
from keras.datasets import cifar10 

In [ ]:
(X_train, y_train), (X_testval, y_testval) = cifar10.load_data()

In [ ]:
# Import dependence for handling arrays
import numpy as np

In [ ]:
# Show the shape of the data partitions
print("X_train original shape:", X_train.shape)
print("y_train original shape:", y_train.shape)
print("X_testval original shape:", X_testval.shape)
print("y_testval original shape:", y_testval.shape)

In [ ]:
# Show the data type of the data partitions
print("X_train original dtype:", X_train.dtype)
print("y_train original dytpe:", y_train.dtype)
print("X_testval original dtype:", X_testval.dtype)
print("y_testval original dtype:", y_testval.dtype)

In [ ]:
# Show the data range of the data partitions
print("X_train original range: [", X_train.min(), ",", X_train.max(), "]")
print("X_testval original range: [", X_train.min(), ",", X_testval.max(), "]")

In [ ]:
# Show the different labels of the data partitions
print("y_train labels: \n", np.unique(y_train))
print("y_testval labels: \n", np.unique(y_testval))

In [ ]:
# Import dependence for visualization of images
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,10)  # Configure figure size for 
                                          # appropriate visualization

In [ ]:
def class_to_string(class_int):
    classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog",
               "horse", "ship", "truck"]
    return classes[class_int]

In [ ]:
# Show 9 images with its respective ground truth labels
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i], cmap='gray', interpolation='none')
    class_str = class_to_string(int(y_train[i]))
    plt.title("Class: " + class_str)

In [ ]:
# Convert the 2D images to 1D array
train_samples = X_train.shape[0]
testval_samples = X_testval.shape[0]
sample_dims = X_train.shape[1] * X_train.shape[2] * X_train.shape[3]  # 32*32*3
X_train_rs = X_train.reshape(train_samples, sample_dims)
X_testval_rs = X_testval.reshape(testval_samples, sample_dims)

In [ ]:
# Show shape of the reshaped dataset
print("Training matrix shape:", X_train_rs.shape)
print("Testing matrix shape:", X_testval_rs.shape)

In [ ]:
# Convert dtype to float32
X_train_fl = X_train_rs.astype('float32')
X_testval_fl = X_testval_rs.astype('float32')

In [ ]:
# Show dtype of the dataset
print("Training matrix dtype:", X_train_fl.dtype)
print("Testing matrix dtype:", X_testval_fl.dtype)

In [ ]:
# Change the range of pixels from [0 255] to [0 1]
X_train_fl /= 255
X_testval_fl /= 255

In [ ]:
# Show the range of pixels
print("Training matrix range:", "[", X_train_fl.min(), ",", X_train_fl.max(), "]")
print("Testing matrix range:", "[", X_testval_fl.min(), ",", X_testval_fl.max(), "]")

In [ ]:
# Import dependence for one-hot encoding
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# One-hot encoding of labels
onehot_enc = OneHotEncoder()
y_train_oh = onehot_enc.fit_transform(y_train.reshape(train_samples, 1)).toarray()
y_testval_oh = onehot_enc.fit_transform(y_testval.reshape(testval_samples, 1)).toarray()

In [ ]:
# Show one-hot encoded labels shape
print("Training one-hot encoded labels shape:", y_train_oh.shape)
print("Testing one-hot encoded labels shape:", y_testval_oh.shape)

In [ ]:
# Divide testval in test and validation partitions
samples_test_nb = int(X_testval.shape[0]/2)
X_val = X_testval_fl[:samples_test_nb]
y_val = y_testval_oh[:samples_test_nb]
X_test = X_testval_fl[samples_test_nb:]
y_test = y_testval_oh[samples_test_nb:]

In [ ]:
# Show shapes of test and validation partitions
print("Validation matrix shape:", X_val.shape)
print("Testing matrix shape:", X_test.shape)

In [ ]:
# Import dependencies for network dessign
from keras.models import Model
from keras.layers import Input, Dense

In [ ]:
# Define the model
input_layer = Input(shape=(X_train_fl.shape[1],))
hidden_layer_1= Dense(128, activation='relu')(input_layer)
hidden_layer_2= Dense(256, activation='relu')(hidden_layer_1)
hidden_layer_3 = Dense(256, activation='relu')(hidden_layer_2)
output_layer = Dense(10, activation='softmax')(hidden_layer_3)
model= Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Show a summary of the model
model.summary()

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train_fl, y_train_oh, epochs=20, batch_size=128,
                    validation_data=(X_val, y_val))

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.ylim(0, 1)
plt.show()

In [ ]:
# Plot training and test loss
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Val'], loc='upper left') 
plt.show()

In [ ]:
# Obtain metrics in the test partition
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Predict test samples post-process them
predicted_classes = model.predict(X_test)
predicted_classes = np.round(predicted_classes)
predicted_classes = np.argmax(predicted_classes, axis=1)

In [ ]:
# Convert test labels to scalars
y_test_scalar = np.argmax(y_test, axis=1)

In [ ]:
# Obtain test samples correctly predicted
correct_indices = np.nonzero(predicted_classes == y_test_scalar)[0]

In [ ]:
# Obtain test samples incorrectly predicted
incorrect_indices = np.nonzero(predicted_classes != y_test_scalar)[0]

In [ ]:
# Show some correctly classified samples
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(32,32,3), cmap='gray', 
               interpolation='none')
    predicted_str = class_to_string(predicted_classes[correct])
    y_test_str = class_to_string(y_test_scalar[correct])
    plt.title("Predicted: " + predicted_str + ", Class: " + y_test_str)

In [ ]:
# Show some incorrectly classified samples
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(32,32,3), cmap='gray', 
               interpolation='none')
    predicted_str = class_to_string(predicted_classes[incorrect])
    y_test_str = class_to_string(y_test_scalar[incorrect])
    plt.title("Predicted: " + predicted_str + ", Class: " + y_test_str)

# 2. CIFAR10 with Convolutional Neural Networks

In [ ]:
# Show the shape of the data partitions
print("X_train original shape:", X_train.shape)
print("y_train one-hot shape:", y_train_oh.shape)

There is no need to reshape since CNN are designed to handle images, what we need to redo is the following:


1.   Change range to [0 1]
2.   Split data in validation and test



In [ ]:
# Change the range of pixels from [0 255] to [0 1]
X_train_fl2 = X_train.astype('float32')
X_testval_fl2 = X_testval.astype('float32')
X_train_fl2 /= 255
X_testval_fl2 /= 255

In [ ]:
# Show the range of pixels
print("Training matrix range:", "[", X_train_fl2.min(), ",", 
      X_train_fl2.max(), "]")
print("Testing matrix range:", "[", X_testval_fl2.min(), ",", 
      X_testval_fl2.max(), "]")

In [ ]:
# Divide testval in test and validation partitions
samples_test_nb = int(X_testval.shape[0]/2)
X_val2 = X_testval_fl2[:samples_test_nb]
y_val2 = y_testval_oh[:samples_test_nb]
X_test2 = X_testval_fl2[samples_test_nb:]
y_test2 = y_testval_oh[samples_test_nb:]

In [ ]:
# Show shapes of test and validation partitions
print("Validation matrix shape:", X_val2.shape)
print("Testing matrix shape:", X_test2.shape)

In [ ]:
# Import depence for CNN
from keras.layers import Conv2D, MaxPool2D, Flatten

In [ ]:
# Define the model
input_layer = Input(shape=(X_train.shape[1],X_train.shape[2], X_train.shape[3]))
conv_layer_1 = Conv2D(filters=8, kernel_size=(3, 3), activation='relu')(input_layer)
maxpool_layer_1 = MaxPool2D(pool_size=(2, 2))(conv_layer_1)
conv_layer_2 = Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(maxpool_layer_1)
maxpool_layer_2 = MaxPool2D(pool_size=(2, 2))(conv_layer_2)
conv_layer_3 = Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(maxpool_layer_2)
flatten_layer = Flatten()(conv_layer_3)
dense_layer = Dense(128, activation='relu')(flatten_layer)
output_layer = Dense(10, activation='softmax')(dense_layer)
model= Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# Show summary of the model
model.summary()

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(X_train_fl2, y_train_oh, epochs=20, batch_size=128,
                    validation_data=(X_val2, y_val2))

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.ylim(0, 1)
plt.show()

In [ ]:
# Plot training and test loss
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Val'], loc='upper left') 
plt.show()

In [ ]:
# Obtain metrics in the test partition
score = model.evaluate(X_test2, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Predict test samples post-process them
predicted_classes = model.predict(X_test2)
predicted_classes = np.round(predicted_classes)
predicted_classes = np.argmax(predicted_classes, axis=1)

In [ ]:
# Convert test labels to scalars
y_test_scalar = np.argmax(y_test, axis=1)

In [ ]:
# Obtain test samples correctly predicted
correct_indices = np.nonzero(predicted_classes == y_test_scalar)[0]

In [ ]:
# Obtain test samples incorrectly predicted
incorrect_indices = np.nonzero(predicted_classes != y_test_scalar)[0]

In [ ]:
# Show some correctly classified samples
plt.figure()
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[correct].reshape(32,32,3), cmap='gray', 
               interpolation='none')
    predicted_str = class_to_string(predicted_classes[correct])
    y_test_str = class_to_string(y_test_scalar[correct])
    plt.title("Predicted: " + predicted_str + ", Class: " + y_test_str)

In [ ]:
# Show some incorrectly classified samples
plt.figure()
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(32,32,3), cmap='gray', 
               interpolation='none')
    predicted_str = class_to_string(predicted_classes[incorrect])
    y_test_str = class_to_string(y_test_scalar[incorrect])
    plt.title("Predicted: " + predicted_str + ", Class: " + y_test_str)

# Exercise 1: Train the CNN model with 50 epochs

In [ ]:
# Train the model


# Exercise 2: Double the number of filters in each Conv layer and re-train

In [ ]:
# Define the model


In [ ]:
# Show summary of the model

In [ ]:
# Compile the model


In [ ]:
# Train the model


# Exercise 3: Train a CNN with 2 Conv Blocks before each Maxpooling hidden layer

In [ ]:
# Define the model


In [ ]:
# Show summary of the model


In [ ]:
# Compile the model


In [ ]:
# Train the model


# Exercise 4: Experiment to improve results